In [ ]:
import pandas as pd
from pprint import pprint
from os import path
from IPython.display import display, HTML
import json
from pandas.io.json import json_normalize
import os
import sqlite3 as db

In [ ]:
#!/usr/bin/env python3

import pandas as pd
from pprint import pprint
from os import path
import json
from pandas.io.json import json_normalize

def loadfile(filename):
	with open(filename) as data_file:
		return json.load(data_file)

def parse(jsonData):
	if jsonData is None:
		raise ValueError("json is none")

	cols = ['timestamp', 'confidence',
			'x', 'y', 'z', 'loc_unit',
			'latitude', 'longitude', 'geo_unit',
			'campus', 'building', 'floor']
	rows = []
	for record in jsonData:
		for notif in record['notifications']:
			rows.append([notif['timestamp'], notif['confidenceFactor'],
						 notif['locationCoordinate']['x'], notif['locationCoordinate']['y'], notif['locationCoordinate']['z'], notif['locationCoordinate']['unit'],
						 notif['geoCoordinate']['latitude'], notif['geoCoordinate']['longitude'], notif['geoCoordinate']['unit'],
						 *notif['locationMapHierarchy'].split('>')[0:3], #Avoid zone, whatever that is
						 ])

	return pd.DataFrame(rows, columns=cols)

def extract_macs(dataframe):
    return list(dataframe.deviceId.unique())

def extract_location_sequence(dataframe, macs):
	pass

def extract_geo(dataframe):
	pass

sqlite_file = 'old_dump.sqlite'
conn = db.connect(sqlite_file)


if __name__ == '__main__':
    workingdir = os.getcwd()
    datadir = os.path.join('..','data')
    
    for file in sorted(os.path.join(workingdir,datadir,f) for f in os.listdir(datadir) if not os.path.isdir(os.path.join(workingdir,datadir,f))):
        print(file)
        data = loadfile(file)
        df = parse(data)
        rows, cols = df.shape
        print(f'Writing {rows} record(s) to db:')
        #for index, row in df.iterrows():
            #print(row[1], row[2], row[3], row[10], row[11])
        df.to_sql('RECORDS', conn, if_exists='append', index=False)
    #parse('../data/notify.json.2019-11-05-05-45')
    